In [13]:
import Runner_Edge

In [14]:
import numpy as np
import Runner_Edge
from time import time
import pickle




class Alex_Net:
    
    def __init__(self):
        self.outputs=[]
        self.layers = []
        self.parameters=[]
        
        self.conv1 = Runner_Edge.PyConvlve2d(3, 96, 11,stride=4)
        self.RelU = Runner_Edge.PyRelU()
        self.max1 = Runner_Edge.PyMaxpool(stride=2,Ksize=3)
        self.conv2 = Runner_Edge.PyConvlve2d(96, 256, 5,Padding=2)
        
        self.conv3 = Runner_Edge.PyConvlve2d(256, 384, 3,Padding=1)
        self.conv4 = Runner_Edge.PyConvlve2d(256, 384, 3,Padding=1)
        self.conv5 = Runner_Edge.PyConvlve2d(256, 256, 3,Padding=1)

        self.fclay1 = Runner_Edge.PyLinear(256*6*6, 100) 
        self.fclay2 = Runner_Edge.PyLinear(100, 10)
        self.softmax=Runner_Edge.PySoftmax()
        self.initialize_parameters()


    def initialize_parameters(self):
        for attrname in dir(self):
            attr = getattr(self, attrname)
            if isinstance(attr, (Runner_Edge.PyConvlve2d, Runner_Edge.PyLinear)):
                self.layers.append(attr)
        self.initial_weights()
    def initial_weights(self):
        self.parameters = [
            (layer.get_weight(), layer.get_bias(), layer.Training_state()) 
            for layer in self.layers
        ]
    

    def __call__(self, *args) :
        
        return self.forward(*args)

    def forward(self, x):
        
        self.outputs=[]
        self.outputs.append(("img", x))
        print(x.shape)

        x = self.conv1(x)
        self.outputs.append((self.conv1, x))
        print(x.shape)

        # x = self.RelU(x)
        # self.outputs.append((self.RelU, x))
        # print(x.shape)

        x = self.max1(x)
        self.outputs.append((self.max1, x))
        print(x.shape)

        x = self.conv2(x)
        self.outputs.append((self.conv2, x))
        print(x.shape)

        # x = self.RelU(x)
        # self.outputs.append((self.RelU, x))
        # print(x.shape)

        x = self.max1(x)
        self.outputs.append((self.max1, x))
        print(x.shape)

        x = self.conv3(x)
        self.outputs.append((self.conv3, x))
        print(x.shape)

        # x = self.RelU(x)
        # self.outputs.append((self.RelU, x))
        # print(x.shape)

        x = self.conv4(x)
        self.outputs.append((self.conv4, x))
        print(x.shape)

        # x = self.RelU(x)
        # self.outputs.append((self.RelU, x))
        # print(x.shape)

        x = self.conv5(x)
        self.outputs.append((self.conv5, x))
        print(x.shape)

        x = self.max1(x)
        self.outputs.append((self.max1, x))
        print(x.shape)

        x = self.fclay1(x)
        self.outputs.append((self.fclay1, x))
        print(x.shape)

        x = self.fclay2(x)
        self.outputs.append((self.fclay2, x))
        print(x.shape)

        x = self.softmax(x)
        self.outputs.append((self.softmax, x))
        print(x.shape)

        return x
    
    def backward(self,y,learning_rate):
        error_layer = self.outputs[-1][0].backward_softmax(self.outputs[-1][1], y)
        self.outputs=self.outputs[:-1]
        instance,output = self.outputs[-1]
        if isinstance(instance, Runner_Edge.PyLinear):
                instance.Backward(error_layer, self.outputs[  -2][1], learning_rate)
                error_layer = instance.grad(error_layer)
        self.outputs=self.outputs[:-1]
        instance,output = self.outputs[-1]
        if isinstance(instance, Runner_Edge.PyLinear):
                instance.Backward(error_layer, self.outputs[  -2][1], learning_rate)
                error_layer = instance.grad(error_layer)
        
        # for i in range(len(self.layers) - 2, -1, -1):
        #     instance,output = self.outputs[i]
        #     if isinstance(instance, Runner_Edge.PyLinear):
        #         instance.Backward(error_layer, self.outputs[i - 1][1], learning_rate)
        #         error_layer = instance.grad(error_layer)
        #     elif isinstance(instance, Runner_Edge.PyConvlve2d):
        #         print(instance)
        #         instance.Backward_conv(error_layer, self.outputs[i - 2][1], learning_rate)
        #         error_layer = instance.grad_conv(error_layer)
        #     elif isinstance(instance,Runner_Edge.PyMaxpool):
        #         error_layer=instance.Grad_maxpool(self.outputs[i - 2][1],self.outputs[i - 1][1],error_layer)
        #     # elif isinstance(instance,Runner_Edge.PyRelU):
        #     #     error_layer = instance.backward(error_layer)  
        self.initialize_parameters()

    def save_model(self, file_path):
    # Create a dictionary to hold the parameters
        architecture = {}
    
        for layer in self.parameters:
        # Extract weights, biases, and training state
            weights = layer[0] if layer[0] is not None else None
            biases = layer[1] if layer[1] is not None else None
            trainable = layer[2]

        # Store in architecture
            architecture[len(architecture)] = {
            'weights': weights,
            'biases': biases,
            'trainable': trainable
            }
    
    # Save the dictionary to a file
        with open(file_path + '.pkl', 'wb') as pkl_file:
            pickle.dump(architecture, pkl_file)
    def load_model(self, file_path):
        with open(file_path + '.pkl', 'rb') as pkl_file:
            architecture = pickle.load(pkl_file)
    
        self.parameters = []
        for layer_info in architecture.values():
            weights = layer_info['weights']
            biases = layer_info['biases']
            trainable = layer_info['trainable']
            self.parameters.append((weights, biases, trainable))


start = time() 
# image=np.random.randn(3,227,227).astype(np.float32)
# # image = image.transpose(2, 0, 1) 
# y=np.zeros(10).astype(np.float32)
# y[2]=1
# cifel=Alex_Net()
# print(cifel.parameters[0][0][0][0])
# output=cifel(image)
# cifel.backward(y,0.01)
# # print(cifel.parameters[0][0][0][0])

print("Time taken:", time() - start)


Time taken: 0.0


In [15]:
class LetNet:
    def __init__(self):
        self.layers = []
        self.parameters=[]
        self.conv1 = Runner_Edge.PyConvlve2d(3, 6, 5,Padding=2)
        self.RelU = Runner_Edge.PyRelU()
        self.max1 = Runner_Edge.PyMaxpool(stride=2,Ksize=2)
        self.conv2 = Runner_Edge.PyConvlve2d(6, 16, 5)
        self.max1 = Runner_Edge.PyMaxpool(stride=2,Ksize=3)
        
        self.fclay1 = Runner_Edge.PyLinear(5*5*16, 120) 
        self.RelU = Runner_Edge.PyRelU()

        self.fclay2 = Runner_Edge.PyLinear(120, 84)
        self.RelU = Runner_Edge.PyRelU()
        self.fclay3 = Runner_Edge.PyLinear(120, 10)

        self.softmax=Runner_Edge.PySoftmax()
        self.layers_with_params = [self.conv1, self.conv2, self.fclay1, self.fclay2, self.fclay3]
        self.initialize_parameters()

    def __call__(self, *args) :
        return self.forward(*args)
    def initialize_parameters(self):
        self.parameters = [
            (layer.get_weight(), layer.get_bias(), layer.Training_state()) 
            for layer in self.layers_with_params
        ]
    def forward(self, x):
        print(x.shape)
        self.layers = []  
        self.layers.append(x)
        
        x = (self.conv1(x)) 
        self.layers.append(x)
        print(x.shape)

        # x=self.RelU(x)
        # self.layers.append(x)
        # print(x.shape)
        x = self.max1(x)                   
        self.layers.append(x)
        print(x.shape)

        x = (self.conv2(x))  
        self.layers.append(x)
        print(x.shape)
        # x=self.RelU(x)
        # self.layers.append(x)
        # print(x.shape)
        x = self.max1(x)                 
        self.layers.append(x)
        print(x.shape)

        x = self.fclay1(x)      
        self.layers.append(x)# T
        print(x.shape)

        x = self.fclay2(x)       
        self.layers.append(x)# T
        print(x.shape)

        
        x = self.fclay3(x)       
        self.layers.append(x)# T
        print(x.shape)

        x=self.softmax(x)
        self.layers.append(x) #T 
        print(x.shape)

        return x
    
    # def backward(self,y,learning_rate):
    #     error_layer = self.softmax.backward_softmax(self.layers[-1], y)
        
    #     for i in range(len(self.layers) - 2, -1, -1):
    #         layer = self.layers[i]
    #         if isinstance(layer, Runner_Edge.PyLinear):
    #             layer.Backward(error_layer, self.layers[i - 1], learning_rate)
    #             error_layer = layer.grad(error_layer)
    #         elif isinstance(layer, Runner_Edge.PyConvlve2d):
    #             layer.Backward_conv(error_layer, self.layers[i - 1], learning_rate)
    #             error_layer = layer.grad_conv(error_layer)
    #         elif isinstance(layer,Runner_Edge.PyMaxpool):
    #             error_layer=layer.Grad_maxpool(self.layers[i - 2],self.layers[i - 1],error_layer)
    #         elif isinstance(layer,Runner_Edge.PyRelU):
    #             error_layer = self.RelU.backward(error_layer, self.layers[i - 1])  
    #     self.initialize_parameters()


    def save_model(self, file_path):
    # Create a dictionary to hold the parameters
        architecture = {}
    
        for layer in self.parameters:
        # Extract weights, biases, and training state
            weights = layer[0] if layer[0] is not None else None
            biases = layer[1] if layer[1] is not None else None
            trainable = layer[2]

        # Store in architecture
            architecture[len(architecture)] = {
            'weights': weights,
            'biases': biases,
            'trainable': trainable
            }
    
    # Save the dictionary to a file
        with open(file_path + '.pkl', 'wb') as pkl_file:
            pickle.dump(architecture, pkl_file)
    def load_model(self, file_path):
        with open(file_path + '.pkl', 'rb') as pkl_file:
            architecture = pickle.load(pkl_file)
    
    # Assuming parameters are the same structure as in save_model
        self.parameters = []
        for layer_info in architecture.values():
            weights = layer_info['weights']
            biases = layer_info['biases']
            trainable = layer_info['trainable']
            self.parameters.append((weights, biases, trainable))


In [16]:
dummy_data = np.random.rand( 3, 224, 224).astype(np.float32)  # 100 images of 32x32x3
print(dummy_data.shape)

conv1 = Runner_Edge.PyConvlve2d(3, 96, 11,stride=4)
c1=(conv1(dummy_data))
print(c1.shape)

max1 = Runner_Edge.PyMaxpool(stride=2,Ksize=3)
m1=(max1(c1))
print(m1.shape)

conv2 = Runner_Edge.PyConvlve2d(96, 256, 5,Padding=2)

c2=conv2(m1)
print(c2.shape)
m2=max1(c2)
print(m2.shape)

conv3 = Runner_Edge.PyConvlve2d(256, 384, 3,Padding=1)

c3=conv3(m2)
print(c3.shape)

conv4 = Runner_Edge.PyConvlve2d(256, 384, 3,Padding=1)
c4=conv4(c3)
print(c4.shape)

conv5 = Runner_Edge.PyConvlve2d(256, 256, 3,Padding=1)
c5=conv5(c4)
print(c5.shape)
m3=max1(c5)
print(m3.shape)

fclay1 = Runner_Edge.PyLinear(256*5*5, 4096) 
f1=fclay1(m3)
print(f1.shape)

fclay2 = Runner_Edge.PyLinear(4096, 1000)
f2=fclay2(f1)
print(f2.shape)

softmax=Runner_Edge.PySoftmax()
s1=softmax(f2)
y=np.zeros(1000).astype(np.float32)
y[1]=1
error_layer=softmax.backward_softmax(y,s1)
fclay2.Backward1d(error_layer,f1,0.01)
error_layer=fclay2.grad(error_layer)
fclay2.Backward3d(error_layer,c5,0.01)
error_layer=fclay1.grad(error_layer)

(3, 224, 224)
(96, 54, 54)
(96, 26, 26)
(256, 26, 26)
(256, 12, 12)
(384, 12, 12)
(384, 12, 12)
(256, 12, 12)
(256, 5, 5)
(4096,)
(1000,)


In [17]:
error_layer.shape

(6400,)

In [18]:
error_layer=max1.Grad_maxpool(c5,m3,error_layer)

In [19]:
np.array(error_layer).shape

(256, 12, 12)

In [20]:
conv5.Backward_conv3d(error_layer,c4,0.01)
error_layer=conv5.Grid_conv3d(error_layer)

In [21]:
error_layer.shape

(256, 14, 14)

In [ ]:
conv4.Backward_conv3d(error_layer,c3,0.01)
error_layer=conv4.Grid_conv3d(error_layer)

: 

In [ ]:
conv4.Backward_conv3d(error_layer,c3,0.01)
# error_layer=conv4.Grid_conv3d(error_layer)

: 

In [ ]:
# Simple test
import numpy as np

# Create dummy data
y=np.zeros((1000,10),dtype=np.float32)
cifel=LetNet()
for i in range (dummy_data.shape[0]):
    start = time() 
    print(cifel.parameters[0][0][0][0])
    output=cifel(dummy_data[0])
    cifel.backward(y[0],0.01)
    print("Time taken:", time() - start)

[[-0.0775294303894043, 0.01912447065114975, -0.0482826791703701, -0.0921575054526329, 0.05778946727514267], [-0.02475801855325699, 0.03365381807088852, 0.05476769059896469, -0.020124606788158417, -0.040277402848005295], [-0.03267989307641983, 0.0671938881278038, -0.013397693634033203, 0.06253945082426071, -0.05592621490359306], [0.08952230960130692, -0.07536398619413376, 0.055908121168613434, -0.0658523216843605, -0.09654969722032547], [-0.0003535747528076172, 0.009883426129817963, 0.0024797767400741577, -0.09173612296581268, -0.04322154074907303]]
Time taken: 0.00601506233215332
[[-0.0775294303894043, 0.01912447065114975, -0.0482826791703701, -0.0921575054526329, 0.05778946727514267], [-0.02475801855325699, 0.03365381807088852, 0.05476769059896469, -0.020124606788158417, -0.040277402848005295], [-0.03267989307641983, 0.0671938881278038, -0.013397693634033203, 0.06253945082426071, -0.05592621490359306], [0.08952230960130692, -0.07536398619413376, 0.055908121168613434, -0.06585232168436

In [ ]:
import cv2
import os 
all=[]
data="E:\\cifar-10\\train"
for img in os.listdir(data):
    all.append(cv2.imread(os.path.join(data,img)))
all=np.array(all,dtype=np.float32)

KeyboardInterrupt: 

In [ ]:
all=np.array(all,dtype=np.float32)
all.shape

(22183, 32, 32, 3)

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

label = pd.read_csv("E:\\cifar-10\\trainLabels.csv")

# Initialize the OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)  # Change `sparse` to `sparse_output` for sklearn 1.2.0+

encoded_labels = encoder.fit_transform(label[['label']])



In [ ]:
x=all[:20000].reshape(20,1000,3,32,32).astype(np.float32)
y=encoded_labels[:20000].reshape(20,1000,10).astype(np.float32)
from tqdm import tqdm


In [ ]:
import albumentations as A
import cv2


In [ ]:
transform =A.Compose([  A.Resize(28,28,p=1),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Rotate(limit=45, p=0.5),
    
    ])


In [ ]:
import matplotlib.pyplot as plt
transformed = transform(image=all[0])["image"]
transformed=transformed.astype(np.float32)
transformed=(transformed-transformed.min())/(transformed.max()-transformed.min())
transformed.shape

(28, 28, 3)

In [ ]:
parameters=[]

In [ ]:
np.array(cifel.parameters[0][0][0][0][0])

array([-0.03630844, -0.03733683,  0.01110848, -0.03265102,  0.09594745])

In [ ]:
at=LetNet()

In [ ]:
for epoch in range(10):
    eopch_acc=0
    for i in (range(x.shape[0])):
        accurcey=0

        for j in tqdm(range(x.shape[1])):

            start = time() 
            image=x[i][j]
            image = image.transpose(2, 0, 1) 
            transformed = transform(image=image)["image"]
            transformed=transformed.astype(np.float32)
            transformed=(transformed-transformed.min())/(transformed.max()-transformed.min())
            output=at(transformed)
            if(np.argmax(output)==np.argmax(y[i][j])):
                    accurcey+=1
            cifel.backward(y[i][j],0.01)
        print(np.array(cifel.parameters[0][0][0][0][0]))
        # print("Time taken:", time() - start)
        print(accurcey)
        eopch_acc+=accurcey
    print(eopch_acc)

100%|██████████| 1000/1000 [00:08<00:00, 115.31it/s]


[-0.03630844 -0.03733683  0.01110848 -0.03265102  0.09594745]
102


100%|██████████| 1000/1000 [00:08<00:00, 115.24it/s]


[-0.03630844 -0.03733683  0.01110848 -0.03265102  0.09594745]
100


  7%|▋         | 72/1000 [00:00<00:09, 102.13it/s]


KeyboardInterrupt: 